In [ ]:
"""
# 1.使用 ollama 的 openhermes 模型
# 2.使用 phidata 的命令行交互模式 assistant.cli_app()
# 3.可以調用 DuckDuckGo 搜尋網路資源
# 4.windows 環境偶而會發生ollama 模型載入失敗的問題，此時需開啟 powershell 視窗輸入 wsl --shutdown 關閉 wsl 環境後重開 ubuntu 環境
# 案例：
# Summarize the top stories on hackernews and reply in traditional chinese.
# Today's Top 10 trending Repositories on GitHub? Introduce them using traditional chinese.
# 爬取 GitHub Trending 頁面,以繁體中文解說前5的項目
# 爬取 GitHub Trending 頁面,以繁體中文解說前5的項目,請使用我已經設定在系統環境變數 GITHUB_API_KEY 來爬取 
# Summarize the top 3 stories on hackernews and translate them into traditional chinese.
# Summarize the top 3 stories on hackernews and reply in traditional chinese. 
# Whats happening in Taiwan today? Summarize top 3 stories with sources and translate to traditional chinese.
# Whats happening in Taiwan today? Summarize top 3 stories with sources and reply in traditional chinese. 
# Write a python script to plot a sine wave and save it to disc as a png file sine_wave.png
# 搜尋 arxiv 有關GPT-4最新的3篇論文,請以繁體中文markdown格式存檔到 arxiv.md  
# 搜尋網路並推薦台中2日遊的景點及行程規劃
"""

In [ ]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# 匯入依賴庫（套件）,assistant 下面有2種
from phi.assistant import Assistant
from phi.assistant.python import PythonAssistant
# llm 有3 大類
from phi.llm.openai import OpenAIChat
from phi.llm.ollama import Ollama
from phi.llm.openai.like import OpenAILike
# 不知道是幹嘛的?
from pydantic import BaseModel, Field

In [ ]:
# 常用的搜尋工具
from phi.tools.duckduckgo import DuckDuckGo

In [ ]:
# 匯入 tools(部分)
from phi.tools.arxiv import ArxivTools
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.email import EmailTools
from phi.tools.file import FileTools
# from phi.tools.google import GoogleTools # 官方尚未完成
from phi.tools.python import PythonTools
from phi.tools.shell import ShellTools
from phi.tools.wikipedia import WikipediaTools
from phi.tools.website import WebsiteTools

In [ ]:
assistant = Assistant(
    llm=Ollama(model="openhermes"),
    tools=[duckdb_tools],
    show_tool_calls=True,
    add_to_system_prompt="""
    Here are the tables you have access to:
    - movies: Contains information about movies from IMDB.
    """,
    debug_mode=True,
)
assistant.print_response("What is the average rating of movies?", markdown=True)

In [ ]:
# gpt-3.5-turbo
assistant = Assistant(
    llm=OpenAIChat(model="gpt-3.5-turbo"),
)

In [ ]:
# LM Studio：base_url="http://localhost:1234/v1"
# freegpt35：base_url="http://localhost:3040/v1"
assistant = Assistant(
    llm=OpenAILike(base_url="http://localhost:3040/v1"),
    tools=[DuckDuckGo()],
    show_tool_calls=True
    )

In [ ]:
# Ollama
# 注意！注意！注意！assistant 加上讓他講中文的 system_promt，會導致無法調用函數的惡果！ 
assistant = Assistant(
    llm=Ollama(model="openhermes"),
    # system_prompt="請一律使用繁體中文回復", # 不可以亂加這個
    tools=[DuckDuckGo()],
    debug_mode=True,
    show_tool_calls=True
)


In [ ]:
# 要讓它吐中文，要放在這裡
assistant.print_response("Whats happening in Taiwan today? Please reply in traditional chinese", markdown=True)
# assistant.print_response("使用DuckDuckGo搜尋網路並推薦台中2日遊的景點及行程規劃", markdown=True)
# assistant.print_response("Share a 2 sentence quick healthy breakfast recipe. You should only use traditional chinese to express", markdown=True)
# assistant.cli_app(markdown=True)

In [ ]:
# 範例 cookbook\ollama\assistant_stream_off.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

assistant = Assistant(
    # llm=Ollama(), # 可以不指定 Ollama 的大模型名稱
    llm=Ollama(model="mistral"),
    description="You help people with their health and fitness goals.",
    debug_mode=True,
)
assistant.print_response("Share a quick healthy breakfast recipe. Reply in traditional chinese.", stream=True, markdown=True)


In [ ]:
# 範例 cookbook\ollama\assistant.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama
# 載入工具
from phi.tools.arxiv import ArxivTools
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.email import EmailTools
from phi.tools.file import FileTools
from phi.tools.google import GoogleTools
from phi.tools.python import PythonTools
from phi.tools.shell import ShellTools
from phi.tools.website import WebsiteTools
from phi.tools.wikipedia import WikipediaTools

assistant = Assistant(
    llm=Ollama(model="openhermes"),
    description="You help people with their health and fitness goals.",
    debug_mode=True,
)
assistant2 = Assistant(
    llm=Ollama(model="openhermes"),
    # system_prompt="請一律使用繁體中文回復",
    debug_mode=True,
    tools=[DuckDuckGo()],
    show_tool_calls=True,
)
# assistant1.print_response("Share 2 quick healthy breakfast recipes.", markdown=True)
assistant2.print_response("Whats happening in Taiwan today? Summarize top 3 stories with sources and reply in traditional chinese.", markdown=True)

In [ ]:
# 範例 cookbook\ollama\auto_assistant.py
# auto assistant 酷喔~~~
# 不過，執行有錯誤，待研究
from phi.assistant import Assistant
from phi.knowledge.pdf import PDFUrlKnowledgeBase
from phi.vectordb.pgvector import PgVector2
from phi.llm.ollama import Ollama
from resources import vector_db

knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://phi-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=PgVector2(
        collection="recipes",
        db_url=vector_db.get_db_connection_local(),
    ),
)
knowledge_base.load(recreate=False)

auto_assistant = Assistant(
    knowledge_base=knowledge_base,
    llm=Ollama(model="openhermes_assistant"),
    # use_tools adds the `search_knowledge_base` and `get_chat_history` functions
    use_tools=True,
    show_tool_calls=True,
)

# will call `search_knowledge_base` to generate this response
auto_assistant.print_response('How do I make pad thai?')
# will call `get_chat_history` to generate this response
auto_assistant.print_response('What was my last question?')

In [ ]:
# 範例 cookbook\ollama\cli.py
# 1.使用 ollama 的 openhermes 模型
# 2.使用 phidata 的命令行交互模式 assistant.cli_app()
# 3.可以調用 DuckDuckGo 搜尋網路資源
# 4.windows 環境偶而會發生 ollama 模型載入失敗的問題，此時需開啟 powershell 視窗輸入 wsl --shutdown 關閉 wsl 環境後重開 ubuntu 環境
# 載入套件
from phi.assistant import Assistant
from phi.llm.ollama import Ollama
from phi.tools.duckduckgo import DuckDuckGo

assistant = Assistant(
    llm=Ollama(model="openhermes_assistant"),
    system_prompt="You ara a helpful assistant.",
	description="You can use provided tools to help users solve any problems. and you should always reply to user in traditional chinese",
    tools=[DuckDuckGo()],
    show_tool_calls=True,
    debug_mode=True
    )
assistant.cli_app(markdown=True)

In [ ]:
# 範例 cookbook\ollama\data_assistant.py
import json
from phi.assistant.duckdb import DuckDbAssistant
from phi.llm.ollama import Ollama

duckdb_assistant = DuckDbAssistant(

    semantic_model=json.dumps({
    
        "tables": [
            {
                "name": "movies",
                "description": "Contains information about movies from IMDB.",
                "path": "https://phidata-public.s3.amazonaws.com/demo_data/IMDB-Movie-Data.csv",
            }
        ]
    }),
    llm=Ollama(model="openhermes_assistant"),
)

duckdb_assistant.print_response("What is the average rating of movies? Show me the SQL.", markdown=True)


In [ ]:
# 範例 cookbook\ollama\embeddings.py
from phi.embedder.ollama import OllamaEmbedder

embedder = OllamaEmbedder(model="nomic-embed-text", dimensions=768)
embeddings = embedder.get_embedding("Embed me")

print(f"Embeddings: {embeddings}")
print(f"Dimensions: {len(embeddings)}")


In [ ]:
# 範例 cookbook\ollama\hermes.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.arxiv import ArxivTools

hermes = Assistant(
    # llm=Ollama(model="openhermes_assistant"), # 很好的模型，可以調用函數
    llm=Ollama(model="openhermes"),
    # llm=Ollama(model="openchat"), # 很遜的模型，不會調用函數
    tools=[DuckDuckGo()],
    show_tool_calls=True,
)
# Search Today's Top 5 trending Repositories on GitHub? Introduce them using traditional chinese
# hermes.print_response("Whats happening in Taiwan? Summarize top stories with sources. 並使用繁體中文回覆", markdown=True)
hermes.print_response("Whats happening in Taiwan? Summarize top 5 social news with sources and at least one new pictures link. 並使用繁體中文回覆", markdown=True)


In [ ]:
# 範例 cookbook\ollama\hn_assistant.py
import json
import httpx

from phi.assistant import Assistant
from phi.llm.ollama import Ollama
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to return. Defaults to 10.

    Returns:
        str: JSON string of top stories.
    """

    # Fetch top story IDs
    response = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json')
    story_ids = response.json()

    # Fetch story details
    stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json')
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        stories.append(story)
    return json.dumps(stories)

assistant = Assistant(
    llm=Ollama(model="openhermes_assistant"),
    tools=[get_top_hackernews_stories],
    debug_mode=True,
    show_tool_calls=True
)
# assistant = Assistant(tools=[get_top_hackernews_stories], show_tool_calls=True)
assistant.print_response("Summarize the top 5 stories and 連結 on hackernews and reply in traditional chinese.", markdown=True)


In [ ]:
# 範例 cookbook\ollama\image.py
from pathlib import Path
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

assistant = Assistant(llm=Ollama(model="llava"))

image_path = Path(__file__).parent / "test_image.jpeg"
assistant.print_response(
    "Whats in the image? 請使用繁體中文回復",
    images=[image_path.read_bytes()],
    markdown=True,
)

In [ ]:
# 範例 cookbook\ollama\openai_api.py
# Please install dependencies using: pip install -U ollama phidata openai
from phi.assistant import Assistant
from phi.llm.ollama.openai import OllamaOpenAI

assistant = Assistant(
    llm=OllamaOpenAI(model="qwen"),
    system_prompt="Who are you and who created you? Respond in 1 sentence. 請一律使用繁體中文回復",
)
assistant.print_response(markdown=True)


In [ ]:
# 範例 cookbook\ollama\pydantic_output.py
# 這個範例有趣，定義一個 MovieScript 的輸出類別，讓 movie_assistant 能夠輸出 MovieScript 類別的實例。
# 讓 gpt 解釋這段程式碼
"""
這段程式碼是一個用於生成電影劇本概念的程式。讓我們逐段來解釋：

導入必要的模塊：
typing.List：用於定義列表類型的類型提示。
pydantic.BaseModel 和 pydantic.Field：用於定義數據模型和模型字段的 Pydantic 库。
rich.pretty.pprint：用於美化打印輸出的 Rich 库中的 pprint 函数。
phi.assistant.Assistant 和 phi.llm.ollama.Ollama：用於生成電影劇本的助手類和 Ollama 大腦模型。

定義 MovieScript 模型：
MovieScript 繼承自 BaseModel，它是一個數據模型，用於描述電影劇本的結構。
有幾個字段，包括 setting（場景）、ending（結局）、genre（類型）、name（名稱）、characters（角色）和 storyline（故事概要）。
每個字段都有一個描述，以便使用者理解。

創建 Assistant 對象：
movie_assistant 是一個助手對象，用於生成電影劇本概念。
通過傳入 Ollama 模型，它使用了一個名為 "openhermes" 的模型來生成劇本概念。
description 參數提供了對助手功能的簡要描述，並要求使用中文詼諧的語氣。
output_model 指定了劇本概念的輸出模型，這裡使用了之前定義的 MovieScript。
debug_mode 設置為 True，以啟用調試模式。

運行 Assistant：
通過調用 movie_assistant.run("First love")，助手開始運行，並提供 "First love" 作為輸入。
pprint 函數用於美化打印輸出。
總之，這段程式碼定義了一個助手，用於生成具有指定場景、結局、類型等元素的電影劇本概念，並且通過 Pydantic 库來驗證輸入和輸出的數據模型。

要求更詳細的解說
導入必要的模塊：
typing.List：這個模塊允許我們使用 List 類型提示，它用於指定一個列表中元素的類型。
pydantic.BaseModel 和 pydantic.Field：Pydantic 是一個用於數據驗證和序列化的庫。在這裡，我們使用它來定義 MovieScript 類型，並添加一些驗證規則，如必填字段和字段描述。
rich.pretty.pprint：Rich 是一個用於在終端中創建美觀且可自定義的文本輸出的庫。pprint 函數用於美化打印輸出的數據。
phi.assistant.Assistant 和 phi.llm.ollama.Ollama：這些是用於生成電影劇本的助手和 Ollama 大腦模型的相關模塊。Assistant 類負責處理用戶輸入並生成相應的劇本概念，而 Ollama 則是一個語言模型，用於生成文本。

定義 MovieScript 模型：
MovieScript 是一個 Pydantic 的數據模型，用於表示電影劇本的結構。
它包含了幾個字段：
setting：表示電影的場景。
ending：表示電影的結局。
genre：表示電影的類型。
name：表示電影的名稱。
characters：表示電影中的角色列表。
storyline：表示電影的故事概要。
每個字段都使用 Field 函數進行了配置，其中 ... 表示該字段是必填的，並且提供了對該字段的描述。

創建 Assistant 對象：
Assistant 類是用於生成劇本概念的助手。在這裡，我們創建了一個名為 movie_assistant 的助手對象。
我們傳入了一個 Ollama 的實例，並指定了使用的模型為 "openhermes"。這個模型將用於生成電影劇本的概念。
我們還提供了一個簡要的描述，以及設置了 debug_mode 為 True，以啟用調試模式。

運行 Assistant：
一切準備就緒後，我們調用 movie_assistant.run("First love") 方法運行助手。
這將生成一個電影劇本概念，並將其打印出來，使用 pprint 函數美化輸出。
這就是這段程式碼的詳細解釋。它的主要功能是生成具有指定場景、結局、類型等元素的電影劇本概念，並通過 Pydantic 库來驗證輸入和輸出的數據模型。

問：typing.List：這個模塊允許我們使用 List 類型提示，它用於指定一個列表中元素的類型。
答：
沒錯，typing 模組中的 List 類型提示允許我們指定列表中元素的類型。這在 Python 中是一種靜態類型提示的方式，它有助於提高程式碼的可讀性和可維護性。
例如，假設我們有一個列表，其中包含多個整數：
my_list = [1, 2, 3, 4, 5]
使用 typing 模組中的 List 類型提示，我們可以指定這個列表的元素類型是整數，如下所示：
from typing import List
my_list: List[int] = [1, 2, 3, 4, 5]
這樣一來，當其他人讀取或維護這段程式碼時，就能清楚地知道 my_list 是一個整數列表，而不需要進一步查看程式碼內容。這種類型提示還可以讓 IDE 在開發過程中提供更好的代碼自動完成和錯誤檢測功能，從而提高開發效率和程式碼的健壯性。

問：
答：
這段程式碼定義了一個名為 MovieScript 的類，它是一個 Pydantic 的數據模型，用於描述電影劇本的結構。讓我們來逐行解釋每個字段：

setting：
這是電影劇本中的場景，它的類型是字串 (str)。
這個字段使用了 Field 函數進行配置，其中 ... 表示該字段是必填的。
提供了對這個字段的描述，指明了使用者應該提供一個適合的場景來描述一部大片。

ending：
這是電影劇本中的結局，它的類型也是字串 (str)。
同樣地，這個字段也是必填的，如果結局沒有被提供，則提供一個快樂的結局。
提供了對這個字段的描述，指明了使用者應該提供電影的結局。

genre：
這是電影劇本中的類型，它的類型是字串 (str)。
同樣地，這個字段也是必填的，如果類型沒有被提供，則從動作、驚悚或浪漫喜劇中選擇一個。
提供了對這個字段的描述，指明了使用者應該提供電影的類型。

name：
這是電影的名稱，它的類型也是字串 (str)。
同樣地，這個字段也是必填的。
提供了對這個字段的描述，指明了使用者應該提供一個以繁體中文表達的名稱。

characters：
這是電影中角色的名稱列表，它的類型是字串列表 (List[str])。
同樣地，這個字段也是必填的。
提供了對這個字段的描述，指明了使用者應該提供以繁體中文表達的角色名稱。

storyline：
這是電影的故事概要，它的類型是字串 (str)。
同樣地，這個字段也是必填的。
提供了對這個字段的描述，指明了使用者應該提供一個以繁體中文表達的令人興奮的三句話故事概要。
總的來說，這個 MovieScript 類型定義了一個完整的電影劇本結構，並且使用 Pydantic 的驗證功能確保了每個字段都被正確填寫。
"""
from typing import List
from pydantic import BaseModel, Field
from rich.pretty import pprint
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

class MovieScript(BaseModel):
    setting: str = Field(..., description="Provide a nice setting for a blockbuster movie.")
    ending: str = Field(..., description="Ending of the movie. If not available, provide a happy ending.")
    genre: str = Field(
        ..., description="Genre of the movie. If not available, select action, thriller or romantic comedy."
    )
    name: str = Field(..., description="Give a name to this movie. express with traditional chinese.")
    characters: List[str] = Field(..., description="Name of characters for this movie. express with traditional chinese.")
    storyline: str = Field(..., description="3 sentence storyline for the movie. Make it exciting! express with traditional chinese.")

movie_assistant = Assistant(
    llm=Ollama(model="openhermes"),
    description="You help people write movie ideas. 請使用中文詼諧的語氣表達",
    output_model=MovieScript,
    # output_model=MovieScript(ending="all died"), # 不能這樣子設，上面定義的類別是給 LLM 輸出用的
    # debug_mode=True,
)

# 給定字串而已，應該是當成提示詞，
# movie_assistant.run ??? 這樣也可以???
pprint(movie_assistant.run("First love"))

# 輸出的部分，endding 沒給，就會是快樂結局，輸出5個元素



In [ ]:
# 範例 cookbook\ollama\PythonAssistant.py
from phi.assistant.python import PythonAssistant
from phi.file.local.csv import CsvFile
# 載入套件
from phi.assistant import Assistant
from phi.llm.ollama import Ollama
from phi.llm.openai.like import OpenAILike

# 加入9個工具之後，一直抱錯，只好試著加入有的沒的
from pydantic import BaseModel, PydanticUserError, field_validator

# 載入工具
from phi.tools.arxiv import ArxivTools
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.email import EmailTools
from phi.tools.file import FileTools
from phi.tools.google import GoogleTools
from phi.tools.python import PythonTools
from phi.tools.shell import ShellTools
from phi.tools.website import WebsiteTools
from phi.tools.wikipedia import WikipediaTools

# 建立函數當成工具使用
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.
    Args:
        num_stories (int): Number of stories to return. Defaults to 10.
    Returns:
        str: JSON string of top stories.
    """
    # Fetch top story IDs
    response = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json')
    story_ids = response.json()

python_assistant = PythonAssistant(
    # llm 都不指定，原本預設使用 gpt4，已修改為使用 gpt3.5-turbo
	# llm=Ollama(model="codellama"),            # 可以跑出結果，但結果錯誤 4.917 
	# llm=Ollama(model="llama2"),               # 6.7、6.4 多次不同結果
	# llm=Ollama(model="llama2-uncensored"),    # 可以跑出結果，但結果錯誤 7.15 
	# llm=Ollama(model="llava"),                # 可以跑出結果，但結果錯誤 6.5
	# llm=Ollama(model="mistral"),              # 任務失敗，輸出空白結果
	# llm=Ollama(model="neural-chat"),          # 可以跑出結果，但結果錯誤 6.9 
	# llm=Ollama(model="openchat"),             # The average rating of movies is not provided in the given data file.
	llm=Ollama(model="openhermes"),           # 可以跑出結果，但結果錯誤 7.1
    # llm=Ollama(model="openhermes_assistant"),
	# llm=Ollama(model="qwen"),                 # 任務失敗，很久還跑出亂碼
	# llm=Ollama(model="starling-lm"),          # 可以跑出結果，但結果錯誤 7.3
	# 這些 ollama 的大模型都沒有成功調用 python 的 tools 來寫程式碼計算平均分數。失敗~
	# 如果使用 gpt-3.5-turbo 幾乎每次都成功
    # 調用 LM Studio
	# llm=OpenAILike(base_url="http://localhost:1234/v1"),
    files=[
        CsvFile(
            path="https://phidata-public.s3.amazonaws.com/demo_data/IMDB-Movie-Data.csv",
            description="Contains information about movies from IMDB.",
        )
    ],
    pip_install=True,
    show_tool_calls=True,

)

python_assistant.print_response("What is the average rating of movies?", markdown=True)



In [ ]:
# 範例 cookbook\ollama\PythonAssistant.py
from phi.assistant.python import PythonAssistant
from phi.file.local.csv import CsvFile
from phi.llm.ollama import Ollama

# 載入工具
from phi.tools.arxiv import ArxivTools
from phi.tools.email import EmailTools
from phi.tools.file import FileTools
from phi.tools.python import PythonTools
from phi.tools.shell import ShellTools
from phi.tools.website import WebsiteTools
from phi.tools.wikipedia import WikipediaTools

# 建立函數當成工具使用
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.
    Args:
        num_stories (int): Number of stories to return. Defaults to 10.
    Returns:
        str: JSON string of top stories.
    """
    # Fetch top story IDs
    response = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json')
    story_ids = response.json()

python_assistant = PythonAssistant(
	llm=Ollama(model="wizardlm2"), 
    files=[
        CsvFile(
            path="https://phidata-public.s3.amazonaws.com/demo_data/IMDB-Movie-Data.csv",
            description="Contains information about movies from IMDB.",
        )
    ],
    pip_install=True,
    show_tool_calls=True,

)

python_assistant.print_response("What is the average rating of movies?", markdown=True)



In [ ]:
# 範例 cookbook\ollama\qwen.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

qwen = Assistant(llm=Ollama(model="qwen"))

qwen.print_response("Give me a short introduction to large language model. 請一律使用繁體中文回復", markdown=True)
qwen.print_response("Write a python function to calculate the factorial of a number. 請一律使用繁體中文回復", markdown=True)



In [ ]:
# 範例 cookbook\ollama\resources.py
"""
Lets create a PDF Assistant that can answer questions from a PDF. We'll use PgVector for knowledge and storage.
Knowledge Base: information that the Assistant can search to improve its responses (uses a vector db).
Storage: provides long term memory for Assistants (uses a database).
Run PgVector
Install docker desktop for running PgVector in a container.
Create a file resources.py with the following contents
"""
from phi.docker.app.postgres import PgVectorDb
from phi.docker.resources import DockerResources

# -*- PgVector running on port 5432:5432
vector_db = PgVectorDb(
    pg_user="ai",
    pg_password="ai",
    pg_database="ai",
    debug_mode=True,
)

# -*- DockerResources
dev_docker_resources = DockerResources(apps=[vector_db])



In [ ]:
# 範例 cookbook\ollama\tool_call.py
import typer
from phi.assistant import Assistant
from phi.tools.duckduckgo import DuckDuckGo
from phi.llm.ollama import Ollama

assistant = Assistant(
    llm=Ollama(model="openhermes"),
    # llm=Ollama(model="qwen"),
    tools=[DuckDuckGo()],
    show_tool_calls=True,
    # instructions=["Make sure your answers are well formatted."],
    instructions=["Make sure your answers are well formatted. 請一律使用繁體中文回復"],
    debug_mode=True,
)
assistant.print_response("Whats happening in AI 前緣科技. 請一律使用繁體中文回復", markdown=False)

assistant.print_response("Tell me about OpenAI Sora", markdown=True)

def tool_call(model: str = "openhermes", debug: bool = False):
    print(f"============= Running: {model} =============")
    Assistant(
        llm=Ollama(model=model),
        tools=[DuckDuckGo()],
        show_tool_calls=True,
        debug_mode=debug,
    ).cli_app(markdown=True)

if __name__ == "__main__":
    typer.run(tool_call)


In [ ]:
# 範例 cookbook\ollama\who_are_you.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

prompt = "Who are you and who created you? Answer in 1 short sentence. 請一律使用繁體中文回復"
temp = 0.3
# models = ["phi", "llava", "llama2", "mixtral", "openhermes", "tinyllama"]
models = ["codellama", "llama2", "llama2-uncensored", "llava", "mistral", "neural-chat", "openchat", "openhermes", "qwen", "starling-lm"]
for model in models:
    print(f"================ {model} ================")
    Assistant(llm=Ollama(model=model, options={"temperature": temp}), system_prompt=prompt).print_response(
        markdown=True
    )

In [ ]:
# 範例 cookbook\openhermes\assistant.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

assistant = Assistant(
    llm=Ollama(model="openhermes"),
    description="You help people with their health and fitness goals.",
)
assistant.print_response("Share a quick healthy breakfast recipe.", markdown=True)


In [ ]:
# 範例 cookbook\openhermes\tool_call.py
from phi.assistant import Assistant
from phi.tools.duckduckgo import DuckDuckGo
from phi.llm.ollama import Ollama

assistant = Assistant(
    llm=Ollama(model="openhermes"),
    tools=[DuckDuckGo()],
    show_tool_calls=True,
    debug_mode=True
)
assistant.print_response("Tell me about OpenAI Sora", markdown=True)


In [ ]:
# 範例 cookbook\openhermes\embeddings.py
from phi.embedder.ollama import OllamaEmbedder

embedder = OllamaEmbedder(model="openhermes", dimensions=4096)
embeddings = embedder.get_embedding("Embed me")

print(f"Embeddings: {embeddings[:10]}")
print(f"Dimensions: {len(embeddings)}")

In [ ]:
# 範例 cookbook\openhermes\data_analyst.py
from phi.assistant import Assistant
from phi.llm.ollama import Ollama
from phi.tools.duckdb import DuckDbTools


duckdb_tools = DuckDbTools(create_tables=False, export_tables=False, summarize_tables=False)
duckdb_tools.create_table_from_path(
    path="https://phidata-public.s3.amazonaws.com/demo_data/IMDB-Movie-Data.csv", table="movies"
)

assistant = Assistant(
    llm=Ollama(model="openhermes"),
    tools=[duckdb_tools],
    show_tool_calls=True,
    add_to_system_prompt="""
    Here are the tables you have access to:
    - movies: Contains information about movies from IMDB.
    """,
    debug_mode=True,
)
assistant.print_response("What is the average rating of movies?", markdown=True)

In [ ]:
# 範例 cookbook\openhermes\pydantic_output.py
from typing import List
from pydantic import BaseModel, Field
from rich.pretty import pprint
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

class MovieScript(BaseModel):
    setting: str = Field(..., description="Provide a nice setting for a blockbuster movie.")
    ending: str = Field(..., description="Ending of the movie. If not available, provide a happy ending.")
    genre: str = Field(
        ..., description="Genre of the movie. If not available, select action, thriller or romantic comedy."
    )
    name: str = Field(..., description="Give a name to this movie")
    characters: List[str] = Field(..., description="Name of characters for this movie.")
    storyline: str = Field(..., description="3 sentence storyline for the movie. Make it exciting!")


movie_assistant = Assistant(
    llm=Ollama(model="openhermes"),
    description="You help people write movie ideas.",
    output_model=MovieScript,
    # debug_mode=True,
)

pprint(movie_assistant.run("TAIWAN"))

In [ ]:
# 自訂函數當成工具
import httpx, json
def get_top_hackernews_stories(num_stories: int = 10) -> str:
    """Use this function to get top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to return. Defaults to 10.

    Returns:
        str: JSON string of top stories.
    """

    # Fetch top story IDs
    response = httpx.get('https://hacker-news.firebaseio.com/v0/topstories.json')
    story_ids = response.json()

    # Fetch story details
    stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json')
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        stories.append(story)
    return json.dumps(stories)

In [ ]:

assistant = PythonAssistant(
    # 沒有支援 gemini
	# llm=Ollama(model="codellama"),            # 可以跑出結果，但結果錯誤 4.917 
	# llm=Ollama(model="llama2"),               # 6.7、6.4 多次不同結果
	# llm=Ollama(model="llama2-uncensored"),    # 可以跑出結果，但結果錯誤 7.15 
	# llm=Ollama(model="llava"),                  # 可以跑出結果，但結果錯誤 6.5
	# llm=Ollama(model="mistral"),               # 任務失敗，沒跑出結果
	# llm=Ollama(model="neural-chat"),            # 可以跑出結果，但結果錯誤 6.9 
	# llm=Ollama(model="openchat"),               # The average rating of movies is not provided in the given data file.
	# llm=Ollama(model="openhermes"),           # 可以跑出結果，但結果錯誤 7.1
    # llm=Ollama(model="openhermes"),
	# llm=Ollama(model="qwen"),                 # 任務失敗，很久還跑出亂碼
	# llm=Ollama(model="starling-lm"),            # 可以跑出結果，但結果錯誤 7.3
	# 這些 ollama 的大模型都沒有成功調用 python 的 tools 來寫程式碼計算平均分數。失敗~
    
	# 如果使用 gpt-3.5-turbo 幾乎每次都成功, 函數調用也沒問題
    llm=OpenAIChat(model="gpt-3.5-turbo"),
	
	# llm=OpenAILike(base_url="http://localhost:1234/v1"),
	
	system_prompt="You ara a helpful assistant.",
	description="You can use provided tools to help users solve any problems.",
	use_tools=True,
	show_tool_calls=True,
	tools=[get_top_hackernews_stories, ArxivTools(), DuckDuckGo(), EmailTools(), FileTools(), PythonTools(), ShellTools(), WebsiteTools(), WikipediaTools()],  # GoogleTools(), 
	pip_install=True,
	save_and_run=True,
	run_code=True,
	debug_mode=True,
	markdown=True
	)


In [ ]:
# Summarize the top stories on hackernews today and translate to traditional chinese.
# Search the top 5 stories on hackernews today. Summarize them and give me the links.
# 搜尋 wikipedia,列出五月天歷年的專輯清單(無需其他說明),寫入檔案 五月天.txt
task="Summarize the top 5 stories on hackernews today. Reply in traditional chinese"

In [ ]:
assistant.print_response(task, markdown=True)
# assistant.cli_app(markdown=False)